In [5]:
sc.hadoopConfiguration.set("fs.s3n.awsAccessKeyId",sys.env("AWS_ACCESS_KEY"))
sc.hadoopConfiguration.set("fs.s3n.awsSecretAccessKey", sys.env("AWS_SECRET_KEY"))

In [6]:
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._

val schema = StructType(
  Array(
    StructField("ip", StringType, true),
    StructField("app", StringType, true),
    StructField("device", StringType, true),
    StructField("os", StringType, true),
    StructField("channel", StringType, true),
    StructField("click_time", StringType, true),
    StructField("attributed_time", StringType, true),
    StructField("is_attributed", StringType, true) 
  )
)

import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
schema: org.apache.spark.sql.types.StructType = StructType(StructField(ip,StringType,true), StructField(app,StringType,true), StructField(device,StringType,true), StructField(os,StringType,true), StructField(channel,StringType,true), StructField(click_time,StringType,true), StructField(attributed_time,StringType,true), StructField(is_attributed,StringType,true))


In [7]:
// data source: https://www.kaggle.com/matleonard/feature-engineering-data
spark.read.format("csv")
    .schema(schema)
    .load("s3n://renan-test/adtracking/*.csv")
    .limit(10).show()

+------+---+------+---+-------+-------------------+-------------------+-------------+
|    ip|app|device| os|channel|         click_time|    attributed_time|is_attributed|
+------+---+------+---+-------+-------------------+-------------------+-------------+
| 89489|  3|     1| 13|    379|2017-11-06 15:13:23|               null|            0|
|204158| 35|     1| 13|     21|2017-11-06 15:41:07|2017-11-07 08:17:19|            1|
|  3437|  6|     1| 13|    459|2017-11-06 15:42:32|               null|            0|
|167543|  3|     1| 13|    379|2017-11-06 15:56:17|               null|            0|
|147509|  3|     1| 13|    379|2017-11-06 15:57:01|               null|            0|
| 71421| 15|     1| 13|    153|2017-11-06 16:00:00|               null|            0|
| 76953| 14|     1| 13|    379|2017-11-06 16:00:01|               null|            0|
|187909|  2|     1| 25|    477|2017-11-06 16:00:01|               null|            0|
|116779|  1|     1|  8|    150|2017-11-06 16:00:01|   

In [ ]:
val trackingStream =  spark.readStream.format("csv")
    .schema(schema)
    .load("s3n://renan-test/adtracking/*.csv")
    .withColumn("value",to_json(struct($"ip",$"app",$"device",$"os",$"channel",$"click_time",$"attributed_time",$"is_attributed")))
    .withColumn("key",$"app")
    .select("key","value")

trackingStream.writeStream
    .format("kafka")
    .option("kafka.bootstrap.servers", "localhost:9092")
    .option("topic", "tracking")
    .option("checkpointLocation", "/home/rhuanca/demo/_checkpoints/tracking")
    .start()
    .awaitTermination()